# Загрузка Pandas и очистка данных

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('files')

Drive already mounted at files; to attempt to forcibly remount, call drive.mount("files", force_remount=True).


In [ ]:
df = pd.read_csv('/content/files/MyDrive/Colab Notebooks/data/Project3/main_task.csv')

In [ ]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [ ]:
new_data_city = pd.get_dummies(df['City'])

In [ ]:
new_data_city.columns

Index(['Amsterdam', 'Athens', 'Barcelona', 'Berlin', 'Bratislava', 'Brussels',
       'Budapest', 'Copenhagen', 'Dublin', 'Edinburgh', 'Geneva', 'Hamburg',
       'Helsinki', 'Krakow', 'Lisbon', 'Ljubljana', 'London', 'Luxembourg',
       'Lyon', 'Madrid', 'Milan', 'Munich', 'Oporto', 'Oslo', 'Paris',
       'Prague', 'Rome', 'Stockholm', 'Vienna', 'Warsaw', 'Zurich'],
      dtype='object')

In [ ]:
new_data_price = pd.get_dummies(df['Price Range'])
new_data_price.columns = ['Low', 'Middle', 'High']

In [ ]:
df = pd.concat([df, new_data_city, new_data_price], axis=1)

In [ ]:
def take_a_last_date(data):
  date = data.split('], [')[1].split(',')[0].strip("'")
  return date if len(date)==10 else None

def take_a_first_date(data):
  date = data.split('], [')[1].split(',')[-1].strip(" ']")
  return date if len(date)==10 else None

df['Last time comment'] = df['Reviews'].apply(take_a_last_date)
df['First time comment'] = df['Reviews'].apply(take_a_first_date)
df['Last time comment'] = pd.to_datetime(df['Last time comment'])
df['First time comment'] = pd.to_datetime(df['First time comment'])

df['Days betwin comments'] = df['Last time comment'] - df['First time comment']
df['Days betwin comments'].fillna(pd.Timedelta(0), inplace=True)
df['Days betwin comments'] = df['Days betwin comments'].dt.days.astype('int16')

In [ ]:
cuisine = set(df['Cuisine Style'].str.cat(sep=',').replace("'", '').replace('[', '').replace(']', '').replace(', ', ',').split(','))

for item in cuisine:
  df[item] = df[df['Cuisine Style'].notna()]['Cuisine Style'].apply(lambda x: 1 if item in x else 0)

def cuisine_count(x):
  return len(x.split(','))

df['Cuisine Style'] = df[df['Cuisine Style'].notna()]['Cuisine Style'].apply(cuisine_count) 
df['Cuisine Style'].fillna(1, inplace=True)
for item in cuisine:
  df[item].fillna(0, inplace=True)

In [ ]:
df['Number of Reviews'].fillna(df['Number of Reviews'].mean(), inplace=True)

In [ ]:
df.info(verbose=True, null_counts=True)

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'City', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA','Rating', 'Last time comment', 'First time comment'], axis = 1)
y = df['Rating']

In [ ]:
X.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 163 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Cuisine Style         40000 non-null  float64
 1   Ranking               40000 non-null  float64
 2   Number of Reviews     40000 non-null  float64
 3   Amsterdam             40000 non-null  uint8  
 4   Athens                40000 non-null  uint8  
 5   Barcelona             40000 non-null  uint8  
 6   Berlin                40000 non-null  uint8  
 7   Bratislava            40000 non-null  uint8  
 8   Brussels              40000 non-null  uint8  
 9   Budapest              40000 non-null  uint8  
 10  Copenhagen            40000 non-null  uint8  
 11  Dublin                40000 non-null  uint8  
 12  Edinburgh             40000 non-null  uint8  
 13  Geneva                40000 non-null  uint8  
 14  Hamburg               40000 non-null  uint8  
 15  Helsinki          

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21455349999999998
